In [81]:
import pandas as pd
import numpy as np

In [2]:
filename = '/ourdisk/hpc/geocarb/vishnupk/folds/xiao_data_12_v1.pkl'
df = pd.read_pickle(filename)

In [6]:
df.head()

,XC,EM,lat,lon
0,"[[[7.6820374, 7.6820383, 7.6826544, 7.682411, ...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[[22.551605, 22.580147, 22.60859, 22.636887, ...","[[[-122.23602, -122.127335, -122.01857, -121.9..."
1,"[[[7.750267, 7.7502675, 7.7518997, 7.751155, 7...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[[22.551605, 22.580147, 22.60859, 22.636887, ...","[[[-122.23602, -122.127335, -122.01857, -121.9..."
2,"[[[7.761965, 7.7619653, 7.7620063, 7.762361, 7...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[[22.551605, 22.580147, 22.60859, 22.636887, ...","[[[-122.23602, -122.127335, -122.01857, -121.9..."
3,"[[[7.769256, 7.769256, 7.7707577, 7.769789, 7....","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[[22.551605, 22.580147, 22.60859, 22.636887, ...","[[[-122.23602, -122.127335, -122.01857, -121.9..."
4,"[[[7.706072, 7.7060895, 7.7054386, 7.704811, 7...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[[22.551605, 22.580147, 22.60859, 22.636887, ...","[[[-122.23602, -122.127335, -122.01857, -121.9..."


In [82]:
X = df['XC'] 
y = df['EM'] 

In [83]:
X = X.to_list()
X = np.ma.getdata(X)
X = X[:,0,:,:]
X = X.astype('float32') 

In [84]:
X_resized = []
for x in X:
    X_resized.append(np.resize(x, (256,256)))

(256, 256)

In [85]:
tres = 15 
y = y.to_list()
y = np.array(y)
y = np.where(y > tres, 1, 0)
y = y.astype('float32')  


In [86]:
# Split data 
test_size = 0.2
test_split = int(len(X)*test_size)

In [87]:
test_split

72

In [94]:
X_new = []
for i in range(5, len(X_resized)):
    X_new.append(np.array(X_resized[i-5:i]))

y_new = y[5:len(y)]

In [99]:
len(X_new)

359

In [100]:
len(y_new)

359

In [49]:
x_train = X_new[:-(2*test_split)]
y_train = y_new[:-(2*test_split)]

x_val = X_new[-(2*test_split):-(test_split)]
y_val = y_new[-(2*test_split):-(test_split)]

x_test = X_new[-(test_split):]
y_test = y_new[-(test_split):]


In [50]:
print(len(x_train))
print(len(x_val))
print(len(x_test))


215
72
72


359

In [28]:
len(X_new)
len(y_new)

359

In [79]:
from sklearn.model_selection import train_test_split
test_size = 0.2 
random_state = 42
x_train , x_rem, y_train, y_rem = train_test_split(X, y, test_size = test_size*2, random_state = random_state) 
    

In [80]:
import tensorflow as tf
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))

In [81]:
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(8)

In [82]:
train_dataset.take(1)

<TakeDataset element_spec=(TensorSpec(shape=(None, 265, 442), dtype=tf.float32, name=None), TensorSpec(shape=(None, 265, 442), dtype=tf.int64, name=None))>